# Защита данных клиентов

С помощью линейного преобразования зашифрованы данные клиентов. 
Теоретически обоснована инвариантность предсказаний линейной регрессии.

## Описание данных

* Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
* Целевой признак: количество страховых выплат клиенту за последние 5 лет.

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Генерируем-матрицу" data-toc-modified-id="Генерируем-матрицу-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Генерируем матрицу</a></span></li><li><span><a href="#Разбиваем-данные,-преобразуем-их" data-toc-modified-id="Разбиваем-данные,-преобразуем-их-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Разбиваем данные, преобразуем их</a></span></li><li><span><a href="#Обучаем-линейную-регрессию-на-преобразованных-и-на-исходных-данных,-считаем-$R^2$" data-toc-modified-id="Обучаем-линейную-регрессию-на-преобразованных-и-на-исходных-данных,-считаем-$R^2$-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Обучаем линейную регрессию на преобразованных и на исходных данных, считаем $R^2$</a></span></li><li><span><a href="#На-всякий-случай-проверим-равенство-$\overline{Xw}-=-Xw$" data-toc-modified-id="На-всякий-случай-проверим-равенство-$\overline{Xw}-=-Xw$-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>На всякий случай проверим равенство $\overline{Xw} = Xw$</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

In [12]:
df = pd.read_csv('/datasets/insurance.csv')
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


__Промежуточный вывод__

Данные не требуют предобработки и готовы к анализу.

## Теоретическая часть

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** 

При преобразовании признаков $X$, посредсвтом умножения на невырожденную матрицу $P$ справа, метрики $MSE$ и $R^2$ линейных регрессий на преобразованных и на исходных данных совпадают.

**Обоснование:** 

Обозначения:

- $\overline X = X \cdot P$ — матрица признаков после преобразования

- $\overline w$ — вектор весов линейной регрессии полученный на преобразованных данных

Вычислим как связан новый вектор $\overline w$ с ветором $w$ и матрицей $P$:

$\overline w = ( \overline X^T \overline X )^{-1} \overline X^T y  = ( P^T X^T X P )^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} (P^T) X^T y = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w$

Тогда:

$$
MSE(\overline{X w}, y) = MSE(X P P^{-1} w, y) = MSE(Xw, y)
$$

Аналогично:

$$
R^2(\overline{X w}, y) = R^2(Xw, y)
$$

Ну и в целом предсказания двух регрессий очевидно будут совпадать из-за равнства $\overline{Xw} = Xw$


Интересно, что если умножить весь сет данных (включая целевой признак) на ортогональную матрицу ($A^T = A^{-1}$) слева, то $\overline w = w$  

В таком случае:

- $\overline X = A \cdot X  $ — матрица признаков после преобразования

- $\overline w$ — вектор весов линейной регрессии полученный на преобразованных данных

- $\overline y = Ay$ — вектор целевого признака после преобразования

$\overline w = ( \overline X^T \overline X )^{-1} \overline X^T  \overline y  = ( X^T A^T A X )^{-1} X^T A^T A y  =(X^T X)^{-1} X^T y = w$

## Алгоритм преобразования

**Алгоритм**

* генерируем случайную матрицу $P$ размера $nxn$, где $n$ - количество признаков (в нашем случае $n = 4$). Коэффициенты сделаем лежащими в диапозоне от 0 до 100. Сгенерировать матрицу с нулевым определителем крайне маловероятно.


* преобразуем данные $\overline X = X \cdot P$, целевой признак не трогаем


* обучаем линейную регрессию на преобразованных данных, считаем $R^2$.


* обучаем линейную регрессию на исходных данных, считаем $R^2$, сравниваем его с предыдущим значением. Метрики должны совпадать

## Проверка алгоритма
### Генерируем матрицу

In [9]:
n = 4
np.random.seed(0)
p = np.random.rand(n, n)
p

array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318],
       [0.4236548 , 0.64589411, 0.43758721, 0.891773  ],
       [0.96366276, 0.38344152, 0.79172504, 0.52889492],
       [0.56804456, 0.92559664, 0.07103606, 0.0871293 ]])

Убедимся в обратимости матрицы:

In [10]:
np.linalg.det(p)

0.1393885146242566

### Разбиваем данные, преобразуем их

In [13]:
features = df.drop(['Страховые выплаты'], axis = 1)
target = df['Страховые выплаты']

In [14]:
new_features = pd.DataFrame (data = features.values @ p, columns = ['f0', 'f1', 'f2', 'f3'])
new_features

,f0,f1,f2,f3
0,47816.159626,19046.821778,39288.176764,26270.382725
1,36639.241064,14601.414441,30105.751495,20139.115638
2,20249.203960,8071.002825,16638.915829,11132.654732
3,40194.769953,16004.926305,33024.265492,22073.819645
4,25164.009197,10026.623866,20676.878699,13829.671933
...,...,...,...,...
4995,34415.758973,13708.798451,28276.978374,18906.692538
4996,50510.900958,20115.221583,41501.340997,27744.501206
4997,32677.776766,13013.436564,26848.372607,17947.547498
4998,31523.345621,12556.239315,25899.851536,17315.289153


### Обучаем линейную регрессию на преобразованных и на исходных данных, считаем $R^2$

In [15]:
model = LinearRegression()
model.fit(new_features, target)
predicted = model.predict(new_features)
print("R2 линейной регресии на преобразованных данных", r2_score(target, predicted) )

R2 линейной регресии на преобразованных данных 0.4249455028666629


In [16]:
model = LinearRegression()
model.fit(features, target)
predicted = model.predict(features)
r2_score(target, predicted)
print("R2 линейной регресии на исходных данных", r2_score(target, predicted))

R2 линейной регресии на исходных данных 0.42494550286668


### На всякий случай проверим равенство $\overline{Xw} = Xw$

In [55]:
model = LinearRegression()
model.fit(new_features, target)
x_w_1 = model.predict(new_features)


model = LinearRegression()
model.fit(features, target)
x_w_2 = model.predict(features)

pd_w =  pd.DataFrame(x_w_1, x_w_2)
pd_w = pd_w.reset_index()
pd_w.columns = ['$X_{new}w_{new}$', '$Xw$']
pd_w

,$X_{new}w_{new}$,$Xw$
0,0.511727,0.511727
1,0.684316,0.684316
2,0.093734,0.093734
3,-0.222589,-0.222589
4,0.065084,0.065084
...,...,...
4995,0.028390,0.028390
4996,0.253367,0.253367
4997,-0.256970,-0.256970
4998,-0.190992,-0.190992


In [56]:
pd_w['$(X_{new}w_{new} - Xw) ^ 2$'] = (pd_w['$X_{new}w_{new}$'] - pd_w['$Xw$']) ** 2
pd_w

,$X_{new}w_{new}$,$Xw$,$(X_{new}w_{new} - Xw) ^ 2$
0,0.511727,0.511727,7.385371e-24
1,0.684316,0.684316,6.642330e-24
2,0.093734,0.093734,5.247921e-24
3,-0.222589,-0.222589,5.825049e-24
4,0.065084,0.065084,7.369690e-24
...,...,...,...
4995,0.028390,0.028390,6.489281e-24
4996,0.253367,0.253367,7.286740e-24
4997,-0.256970,-0.256970,6.027748e-24
4998,-0.190992,-0.190992,7.090876e-24


In [57]:
pd_w['$(X_{new}w_{new} - Xw) ^ 2$'].sum()

3.3772000702627354e-20

Все верно!)

## Общий вывод

Теоретические выкладки полностью подтвердились на практике. На проебразованных и исходных данных метрики линейной регрессии совпадают после линейного преобразования признаков.